In [72]:
from re import sub

from ecomm import Postgres
from pandas import DataFrame, isna
from numpy import nan
from tqdm import tqdm

In [73]:
_df_aplic: DataFrame = DataFrame()
_df_aplic_orig: DataFrame = DataFrame(columns=['num_orig', 'aplic'])

In [82]:
# def read_apl():
with Postgres() as db:
    with open('./data/apl.sql', 'r', encoding='utf-8') as fp:
        _df_aplic: DataFrame = db.query(fp.read())
        _df_aplic['ano'] = None
        _df_aplic_orig = db.query('''
                SELECT original.nu_origina as num_orig,
                    original.codpro,
                    original.num_orig as original_ref
                FROM "D-1".original
            ''')

In [75]:
# def criar_ano_inicial_ano_final():
lista_temp = []
for idx in tqdm(_df_aplic.index):
    row = _df_aplic.loc[idx]
    ano_i = row['ano_i']
    ano_f = row['ano_f']
    if not isna(ano_f) and isna(ano_i):
        lista_temp.append(f'até {ano_f}')
    elif not isna(ano_i) and isna(ano_f):
        lista_temp.append(f'de {ano_i} em diante')
    elif not isna(ano_i) and not isna(ano_f):
        lista_temp.append(f'{ano_i} até {ano_f}')
    elif isna(ano_i) and isna(ano_f):
        lista_temp.append('(Confira o ano de veículo no campo de perguntas.)')
    else:
        lista_temp.append(nan)
_df_aplic.loc[:, 'ano'] = lista_temp

100%|██████████| 126059/126059 [00:04<00:00, 30623.01it/s]


In [76]:
def get_aplicacao(original: str) -> str:
    df_temp = _df_aplic.query(
        'num_orig == @original'
    ).drop(['ano_i', 'ano_f'], axis=1).reset_index(drop=True).copy()
    string_base = ''
    for idx in df_temp.index:
        aplic = ''
        for elemento in df_temp.iloc[idx, 1:]:
            if elemento is None or elemento == '':
                continue
            aplic += (' ' + elemento)
        string_base += f'• {aplic}\n' if len(df_temp) > 1 else f'• {aplic}'
    return string_base

In [90]:
def lista_originais(original: str):
    lista_oem = _df_aplic_orig.query('num_orig == @oem').original_ref.to_list()
    return ', '.join(lista_oem)

In [89]:
lista_originais('M4LT60435')

ZM996, Z70996


In [77]:
def criar_descricao(
        codpro: str,
        aplicacao_veiculos: str,
        titulo: str,
        marca: str,
        num_fab: str,
        oems: list[str],
        multiplo_venda: int
    ) -> str:

    veiculos = f'\nSERVE NOS SEGUINTES VEÍCULOS:\n(Em caso de dúvidas, perguntar no campo de perguntas.)\n\n{aplicacao_veiculos}\n\n' \
        if not isna(aplicacao_veiculos) else '\n(Em caso de dúvidas, perguntar no campo de perguntas.)\n\n'
    descricao = \
f'''{titulo}
{veiculos}
CARACTERÍSTICAS DO PRODUTO:
• Marca: {marca}
• Código da peça: {num_fab}
• Código de referência: {', '.join(oems.split(','))}

CONTEÚDO DA EMBALAGEM:
• {multiplo_venda} {'Unidade' if multiplo_venda == 1 else 'Unidades'}

OBSERVAÇÃO:
• A Responsabilidade de confirmar a aplicação no veículo é exclusivamente do proprietário e do mecânico, uma vez que não temos acesso pessoal e visual da peça instalada.
• Nossos produtos tem garantia de fábrica de 3 meses. Não cobrimos má instalação ou mau uso do produto, recomendamos que a instalação seja feita por um profissional especializado.
• As compras realizadas em nome de Pessoa Jurídica podem estar sujeitas à cobrança de ICMS e DIFAL, conforme Protocolo ICMS 41, de 4 de Abril de 2008. Caso você tenha dúvidas sobre o percentual a ser aplicado, consulte a Cláusula Segunda, §1° do referido Protocolo.
• Para devolução por Defeito de Fabricação o produto deve acompanhar a Nota Fiscal de compra para sua identificação. Sem ela (Nota Fiscal), não é possível identificar o solicitante da Garantia
• O produto deve ser devolvido nas mesmas condições em que foi enviado (na embalagem original, sem sinais de utilização para a perfeita condição de uso do próximo comprador).
• Caixa (s) e Imagem (s) Ilustrativa (s), meramente comercial para efeito estético e informativo de marca e modelo do anúncio.

NÚMERO INTERNO:
• {codpro}'''
    return descricao

In [81]:
get_aplicacao(original='M4LT60435')

''

In [79]:
_df_aplic_orig.query('num_orig == "M4LT60435"')

,num_orig,aplic


In [80]:
criar_descricao(
    codpro='087449',
    aplicacao_veiculos=get_aplicacao(original='M4LT60435'),
    titulo='',
    marca=
)

TypeError: criar_descricao() missing 6 required positional arguments: 'aplicacao_veiculos', 'titulo', 'marca', 'num_fab', 'oems', and 'multiplo_venda'